# FORESEE - Massive spin-2 mediator

### Load Libraries 

In [ ]:
import numpy as np
import sys
import os

src_path = "../../src"
sys.path.append(src_path)
from foresee import Foresee, Utility, Model

from main import sigma_gNucleus_G2Nucleus_analyt, sigma_G2Nucleus_gNucleus_analyt, sigma_ge_G2e_analyt, sigma_G2e_ge_analyt
from constants import *


from matplotlib import pyplot as plt
import matplotlib.tri as tri

plt.rc('text', usetex=True)
plt.rcParams['figure.dpi'] = 400

plt.rcParams['text.latex.preamble'] = [r"\usepackage{amsmath}"]
plt.rcParams['text.latex.preamble'] = [r"\usepackage{amssymb}"]
plt.rcParams['text.latex.preamble'] = [r"\usepackage{siunitx}"]
font = {'family': 'serif', 'serif': ['computer modern roman']}

plt.rc('font', **font)

SMALL_SIZE = 10
MEDIUM_SIZE = 14
BIGGER_SIZE = 16

plt.rc('font', size=MEDIUM_SIZE)  # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)  # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)  # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)  # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

## 1. Initialization 

In [ ]:
foresee = Foresee()

In [ ]:
energy="0.069"

modelname = "Spin2_massive"
model = Model(modelname)

In [ ]:
masses = [float(x) for x in ['{:0.4e}'.format(m) for m in np.logspace(-2,np.log10(0.5),21)]]

num_of_masses = len(masses)

model.add_production_direct(
    label = "NuCal_Prim",
    energy = energy,
    coupling_ref=1,
    masses=masses,
)

In [ ]:
model.set_ctau_1d(filename="model/ctau_G2.txt", coupling_ref=1)

branchings = [
    [ "BR_G2_gg", "black", "solid", r"$\gamma\gamma$", 0.110, 0.30 ],  
    [ "BR_G2_ee", "red", "solid",  r"$e^+ e^-$", 0.110, 0.016],
    [ "BR_G2_mumu", "green", "solid",  r"$\mu^+ \mu^-$", 0.210, 0.016],
]

model.set_br_1d(
    modes=[channel for channel, _, _, _, _, _ in branchings],
    filenames=[ "model/br/" + channel + ".txt" for channel, _, _, _, _, _ in branchings ], )

foresee.set_model(model=model)

## 3. Generate LLP Spectra

In [ ]:
# plt, _ = foresee.get_llp_spectrum(mass=0.1, coupling=1e-4, do_plot=True, save_file=False)
# plt.show()

In [ ]:
from timeit import default_timer as timer

for count, mass in enumerate(masses):
    mass_llp0 = 0.0    # photon
    mass_llp1 = mass   # G2

    start = timer()
    
    foresee.get_llp_spectrum(mass=mass_llp1, mass_llp0=mass_llp0, coupling=1, detector="NuCal", stat_cuts_llp0="p_llp0.e>1.0", stat_cuts_llp1="p_llp1.e>1.0")

    end = timer()
    time_length_sec = end - start

    count += 1
    time_length_sec_total = time_length_sec * num_of_masses
    print("%.2f%% done, " % float(count / num_of_masses * 100), "approx. total run time : %.1f m, " % float(time_length_sec_total / 60), "approx. waiting time: %.1f m" % float(time_length_sec_total * (1 - count / num_of_masses) / 60))

## 5. Parameter Scan

In [ ]:
num_of_couplings = 42

luminosity, distance = 1/1000, 64
setup, selection, channels, length = "NuCal_gg", "np.sqrt(x.x**2 + x.y**2) >= 0", [ "BR_G2_gg" ], 23
foresee.set_detector(length=length, selection=selection, channels=channels, distance=distance, luminosity=luminosity)

list_nevents = []    
for mass in masses:
    couplings, _, nevents, _, _  = foresee.get_events(mass=mass, energy=energy, couplings=np.logspace(-8, -2, num_of_couplings), detector="NuCal", preselectioncuts="p>10")
    list_nevents.append(nevents)  
        
#save results
np.save("model/results/"+energy+"TeV_"+setup+".npy",[masses,couplings,list_nevents])

In [ ]:
num_of_couplings = 42

luminosity, distance = 1/1000, 64
setup, selection, channels, length = "NuCal_ee", "np.sqrt(x.x**2 + x.y**2) >= 0", [ "BR_G2_ee" ], 23
foresee.set_detector(length=length, selection=selection, channels=channels, distance=distance, luminosity=luminosity)

list_nevents = []    
for mass in masses:
    couplings, _, nevents, _, _  = foresee.get_events(mass=mass, energy=energy, couplings=np.logspace(-8, -2, num_of_couplings), detector="NuCal", preselectioncuts="p>10")
    list_nevents.append(nevents)  
        
#save results
np.save("model/results/"+energy+"TeV_"+setup+".npy",[masses,couplings,list_nevents])